# It's a Machine and Natural Language Tagger

In [1]:
from src.IaMaN.base import LM
from src.utils.data import load_ud
from collections import defaultdict
from collections import Counter
from tqdm import tqdm
import numpy as np
import os, re
##
seed = 691; max_char = 200_000_000
m = 10; space = False; fine_tune = True; num_posttrain = 5000
positional = 'dependent'; positionally_encode = True; bits = 50
do_ife = False; update_ife = False; update_bow = True
runners = 25; gpu = True; tokenizer = 'hr-bpe'
##
##
print("Loading post-training data...")
posttrain_path = '/local-data/newstweet/week_2019-40_article_texts/'
total_posttrain = len([posttrain_file for posttrain_file in os.listdir(posttrain_path) if re.search("^\d+.txt$", posttrain_file)])
all_posttrain_files = [posttrain_file for posttrain_file in os.listdir(posttrain_path) if re.search("^\d+.txt$", posttrain_file)]
if num_posttrain:
    np.random.seed(seed)
    posttrain_files = np.random.choice(all_posttrain_files, size=num_posttrain, replace=False)
else:
    posttrain_files = np.array([])
ptdocs = [[[open(posttrain_path+posttrain_file).read()]] for posttrain_file in tqdm(posttrain_files)]
print("Loading gold-tagged UDs data...")
load_set = "GUM"
all_docs = load_ud("English", num_articles = 0, seed = seed, load_set = load_set, rebuild = True, space = space)
test_docs = [doc for doc in all_docs if 'test' in doc['id'] and len(doc['text']) <= max_char]# [:2]
train_docs = [doc for doc in all_docs if 'test' not in doc['id'] and len(doc['text']) <= max_char]# [:10]
nsamp = len(test_docs)
##
print('Avail. post-train, total post-train, Avail. gold, total gold-train, total test-gold: ', 
      total_posttrain, len(ptdocs), len(all_docs), len(train_docs), len(test_docs))

100%|██████████| 5000/5000 [00:00<00:00, 31991.31it/s]

Loading post-training data...
Loading gold-tagged UDs data...


Avail. post-train, total post-train, Avail. gold, total gold-train, total test-gold:  14198 5000 150 132 18


In [2]:
docs = [["".join([row[1] for row in s]) for s in d['conllu']] for d in train_docs]
tdocs = [["".join([row[1] for row in s]) for s in d['conllu']] for d in test_docs]
covering = [[[row[1] for row in s] for s in d['conllu']] for d in train_docs]
tcovering = [[[row[1] for row in s] for s in d['conllu']] for d in test_docs]
covering_vocab = set([t for d in covering for s in d for t in s])

all_layers = {d_i: {# 'lem': [[row[2] for row in s] for s in d['conllu']], # note: for speed, remove lemma layer
                    'sty': [[d['s_type'][s_i] for row in s] for s_i, s in enumerate(d['conllu'])], 
                    'pos': [[row[3] for row in s] for s in d['conllu']], 
                    'sup': [[(str(int(row[6]) - int(row[0])) if int(row[6]) else row[6]) for row in s] for s in d['conllu']], 
                    'dep': [[row[7] for row in s] for s in d['conllu']]}
              for d_i, d in enumerate(train_docs)}

model = LM(m = m, tokenizer = tokenizer, noise = 0.001, seed = seed, space = space, positional = positional,
           positionally_encode = positionally_encode, do_ife = do_ife, runners = runners, gpu = gpu, bits = bits)
data_streams = model.fit(docs, f'{load_set}-{nsamp}', covering = covering, all_layers = all_layers, fine_tune = fine_tune)
model.post_train(ptdocs, update_ife = update_ife, fine_tune = fine_tune)
if fine_tune and ptdocs:
    model.fine_tune(docs, covering = covering, all_layers = all_layers, streams = data_streams)

Initializing:   7%|▋         | 467/6503 [00:00<00:01, 4669.23it/s]

Training tokenizer...


  1%|          | 1/132 [00:00<00:15,  8.22it/s]

Built a vocabulary of 10703 types
Tokenizing documents...



100%|██████████| 132/132 [00:14<00:00,  8.81it/s]


...

100%|██████████| 12/12 [00:21<00:00,  1.83s/it][A

  0%|          | 0/12 [00:00<?, ?it/s]

Re-building dense output heads...



100%|██████████| 12/12 [00:20<00:00,  1.71s/it][A

  0%|          | 0/132 [00:00<?, ?it/s]

Done.
Model params, types, encoding size, contexts, vec dim, max sent, and % capacity used: 52500 12225 50 1050 268 101 0.409
Fine-tuning dense output heads...



100%|██████████| 132/132 [04:28<00:00,  2.03s/it]


In [3]:
interpret_docs = list([docs[3][0:1]])
print(interpret_docs)
model.interpret(interpret_docs, seed = 691)
for doc in model._documents:
    print('opening next doc:')
    for s in doc._sentences:
        print(f'opening next sent: {s._sty}')
        for t in s._tokens:
            print(f'opening next token: {t._form}, {t._lem}, {t._sep}, {t._pos}, {t._sup}, {t._dep}')

[['Emperor Norton ']]



0it [00:00, ?it/s]
1it [00:02,  2.88s/it]

100%|██████████| 1/1 [00:00<00:00, 31.72it/s]

Interpreting documents...
opening next doc:
opening next sent: decl
opening next token: Emperor Norto, None, False, NOUN, 0, root
opening next token: n , None, True, VERB, -1, xcomp


In [4]:
interpret_docs = list([tdocs[0][1:2]])
print(interpret_docs)
model.interpret(interpret_docs, seed = 691)
for doc in model._documents:
    print('opening next doc:')
    for s in doc._sentences:
        print(f'opening next sent: {s._sty}')
        for t in s._tokens:
            print(f'opening next token: {t._form}, {t._lem}, {t._sep}, {t._pos}, {t._sup}, {t._dep}')

[['Results from a nationally representative sample of adults ']]



0it [00:00, ?it/s]
1it [00:02,  2.86s/it]

100%|██████████| 1/1 [00:00<00:00, 11.15it/s]

Interpreting documents...
opening next doc:
opening next sent: decl
opening next token: Results, None, False, NOUN, 1, punct
opening next token:  from, None, False, ADP, 1, case
opening next token:  a, None, False, DET, 3, det
opening next token:  natio, None, False, ADJ, 1, amod
opening next token: n, None, False, NOUN, 1, nmod
opening next token: ally representative sample, None, False, NOUN, 0, root
opening next token:  of, None, False, ADP, -1, case
opening next token:  adults , None, True, DET, -2, nmod


In [5]:
interpret_docs = list([tdocs[0][1:2]])
print(interpret_docs)
interpret_covering =  [[[row[1] for row in s] for s in d['conllu']][1:2] for d in test_docs[0:1]]
print(interpret_covering)
model.interpret(interpret_docs, seed = 691, covering = interpret_covering)

accuracy = defaultdict(list)
accuracy_nsp = defaultdict(list)
accuracy_all, accuracy_all_nsp, = [], []
sup_accuracy, sup_accuracy_nsp, = 0, 0
accuracy_sty = defaultdict(list)
accuracy_all_sty = []

for d_i, doc in enumerate(model._documents):
    for s_i, s in enumerate(doc._sentences):
        if s._sty is not None:
            result = s._sty == test_docs[0:1][d_i]['s_type'][s_i+1]
            accuracy_sty[test_docs[0:1][d_i]['s_type'][s_i+1]].append(result)
            accuracy_all_sty.append(result)

pred_toks = [t._form for doc in model._documents for s in doc._sentences for t in s._tokens]
pred_arcs = set([(ix, str(t._sup), t._dep) for doc in model._documents for s in doc._sentences for ix, t in enumerate(s._tokens)])
pred_spans = list(np.cumsum([len(t) for t in pred_toks]))
pred_stream = [t._pos for doc in model._documents for s in doc._sentences for t in s._tokens]
pred_spans = {(sh-len(gt), sh): (gl, gt)
              for sh, gt, gl in zip(pred_spans, pred_toks, pred_stream)}

gold_toks = [row[1] for d in test_docs[:1] for s in d['conllu'][1:2] for row in s]
gold_arcs = set([(ix, (str(int(row[6]) - int(row[0])) if int(row[6]) else row[6]),
                  row[7]) for d in test_docs[:1] for s in d['conllu'][1:2] for ix, row in enumerate(s)])
gold_spans = list(np.cumsum([len(t) for t in gold_toks]))
gold_stream = [row[3] for d in test_docs[:1] for s in d['conllu'][1:2] for row in s]
gold_spans = {(sh-len(gt), sh): (gl, gt)
              for sh, gt, gl in zip(gold_spans, gold_toks, gold_stream)}

for gold_span in gold_spans:
    if gold_span in pred_spans:
        result = gold_spans[gold_span] == pred_spans[gold_span]
    else:
        result = False
    accuracy[gold_spans[gold_span][0]].append(result)
    accuracy_all.append(result)
    if gold_spans[gold_span][1] != ' ':
        accuracy_nsp[gold_spans[gold_span][0]].append(result)
        accuracy_all_nsp.append(result)
        
for ptok, parc in zip(pred_toks, pred_arcs):
    if parc in gold_arcs:
        sup_accuracy += 1
        if ptok != ' ':
            sup_accuracy_nsp += 1
sup_accuracy /= len(pred_toks)
sup_accuracy_nsp /= len([x for x in pred_toks if x != ' '])

print("Tag-wise POS accuracy with/out space", {tag: sum(accuracy[tag])/len(accuracy[tag]) for tag in accuracy}, 
                                          {tag: sum(accuracy_nsp[tag])/len(accuracy_nsp[tag]) for tag in accuracy_nsp})
print("Overall POS accuracy with/out space", sum(accuracy_all)/len(accuracy_all), sum(accuracy_all_nsp)/len(accuracy_all_nsp))
print("Overall SUP:DEP accuracy with/out space", sup_accuracy, sup_accuracy_nsp)
if len(accuracy_all_sty):
    print("Overall s_type accuracy: ", sum(accuracy_all_sty)/len(accuracy_all_sty))
print("Tag-wise accuracy", list(Counter({tag: (sum(accuracy_sty[tag])/len(accuracy_sty[tag]), len(accuracy_sty[tag])) 
                                         for tag in accuracy_sty}).most_common()))

[['Results from a nationally representative sample of adults ']]
[[['Results', ' from', ' a', ' nationally', ' representative', ' sample', ' of', ' adults ']]]



0it [00:00, ?it/s]
1it [00:03,  3.10s/it]

100%|██████████| 1/1 [00:00<00:00, 11.84it/s]

Interpreting documents...
Tag-wise POS accuracy with/out space {'NOUN': 0.6666666666666666, 'ADP': 1.0, 'DET': 1.0, 'ADV': 0.0, 'ADJ': 0.0} {'NOUN': 0.6666666666666666, 'ADP': 1.0, 'DET': 1.0, 'ADV': 0.0, 'ADJ': 0.0}
Overall POS accuracy with/out space 0.625 0.625
Overall SUP:DEP accuracy with/out space 0.25 0.25
Overall s_type accuracy:  0.0
Tag-wise accuracy [('frag', (0.0, 1))]


In [6]:
interpret_docs = list([tdocs[0][:2]])
print(interpret_docs)
interpret_covering =  [[[row[1] for row in s] for s in d['conllu']][:2] for d in test_docs[0:1]]
print(interpret_covering)
model.interpret(interpret_docs, seed = 691, covering = interpret_covering)

accuracy = defaultdict(list)
accuracy_nsp = defaultdict(list)
accuracy_all, accuracy_all_nsp, = [], []
sup_accuracy, sup_accuracy_nsp, = 0, 0
accuracy_sty = defaultdict(list)
accuracy_all_sty = []

for d_i, doc in enumerate(model._documents):
    for s_i, s in enumerate(doc._sentences):
        if s._sty is not None:
            result = s._sty == test_docs[0:1][d_i]['s_type'][s_i]
            accuracy_sty[test_docs[0:1][d_i]['s_type'][s_i]].append(result)
            accuracy_all_sty.append(result)

pred_toks = [t._form for doc in model._documents for s in doc._sentences for t in s._tokens]
pred_arcs = set([(ix, str(t._sup), t._dep, s_i, d_i) for d_i, doc in enumerate(model._documents) for s_i, s in enumerate(doc._sentences) for ix, t in enumerate(s._tokens)])
pred_spans = list(np.cumsum([len(t) for t in pred_toks]))
pred_stream = [t._pos for doc in model._documents for s in doc._sentences for t in s._tokens]
pred_spans = {(sh-len(gt), sh): (gl, gt)
              for sh, gt, gl in zip(pred_spans, pred_toks, pred_stream)}

gold_toks = [row[1] for d in test_docs[:1] for s in d['conllu'][:2] for row in s]
gold_arcs = set([(ix, (str(int(row[6]) - int(row[0])) if int(row[6]) else row[6]), row[7], s_i, d_i) 
                 for d_i, d in enumerate(test_docs[:1]) for s_i, s in enumerate(d['conllu'][:2]) for ix, row in enumerate(s)])
gold_spans = list(np.cumsum([len(t) for t in gold_toks]))
gold_stream = [row[3] for d in test_docs[:1] for s in d['conllu'][:2] for row in s]
gold_spans = {(sh-len(gt), sh): (gl, gt)
              for sh, gt, gl in zip(gold_spans, gold_toks, gold_stream)}

for gold_span in gold_spans:
    if gold_span in pred_spans:
        result = gold_spans[gold_span] == pred_spans[gold_span]
    else:
        result = False
    accuracy[gold_spans[gold_span][0]].append(result)
    accuracy_all.append(result)
    if gold_spans[gold_span][1] != ' ':
        accuracy_nsp[gold_spans[gold_span][0]].append(result)
        accuracy_all_nsp.append(result)
        
for ptok, parc in zip(pred_toks, pred_arcs):
    if parc in gold_arcs:
        sup_accuracy += 1
        if ptok != ' ':
            sup_accuracy_nsp += 1
sup_accuracy /= len(pred_toks)
sup_accuracy_nsp /= len([x for x in pred_toks if x != ' '])

print("Tag-wise POS accuracy with/out space", {tag: sum(accuracy[tag])/len(accuracy[tag]) for tag in accuracy}, 
                                          {tag: sum(accuracy_nsp[tag])/len(accuracy_nsp[tag]) for tag in accuracy_nsp})
print("Overall POS accuracy with/out space", sum(accuracy_all)/len(accuracy_all), sum(accuracy_all_nsp)/len(accuracy_all_nsp))
print("Overall SUP:DEP accuracy with/out space", sup_accuracy, sup_accuracy_nsp)
if len(accuracy_all_sty):
    print("Overall s_type accuracy: ", sum(accuracy_all_sty)/len(accuracy_all_sty))
print("Tag-wise accuracy", list(Counter({tag: (sum(accuracy_sty[tag])/len(accuracy_sty[tag]), len(accuracy_sty[tag])) 
                                         for tag in accuracy_sty}).most_common()))

[['The prevalence of discrimination across racial groups in contemporary America: ', 'Results from a nationally representative sample of adults ']]
[[['The', ' prevalence', ' of', ' discrimination', ' across', ' racial', ' groups', ' in', ' contemporary', ' America', ': '], ['Results', ' from', ' a', ' nationally', ' representative', ' sample', ' of', ' adults ']]]



0it [00:00, ?it/s]
1it [00:02,  2.79s/it]

100%|██████████| 1/1 [00:00<00:00,  5.41it/s]

Interpreting documents...
Tag-wise POS accuracy with/out space {'DET': 1.0, 'NOUN': 0.5, 'ADP': 0.8, 'ADJ': 0.0, 'PROPN': 0.0, 'PUNCT': 0.0, 'ADV': 0.0} {'DET': 1.0, 'NOUN': 0.5, 'ADP': 0.8, 'ADJ': 0.0, 'PROPN': 0.0, 'PUNCT': 0.0, 'ADV': 0.0}
Overall POS accuracy with/out space 0.47368421052631576 0.47368421052631576
Overall SUP:DEP accuracy with/out space 0.15789473684210525 0.15789473684210525
Overall s_type accuracy:  0.0
Tag-wise accuracy [('frag', (0.0, 2))]


In [7]:
from tqdm import tqdm

confusion = Counter()
confusion_nsp = Counter()
accuracy = defaultdict(list)
accuracy_nsp = defaultdict(list)
accuracy_all, accuracy_all_nsp, = [], []
sup_accuracy, sup_accuracy_nsp, = 0, 0
accuracy_sty = defaultdict(list)
accuracy_all_sty = []

model.interpret(tdocs, seed = 691) 

pred_toks = [t._form for doc in model._documents for s in doc._sentences for t in s._tokens]
pred_spans = list(np.cumsum([len(t) for t in pred_toks]))
pred_spans = set([(sh-len(gt), sh) for sh, gt in zip(pred_spans, pred_toks)])

gold_toks = [row[1] for d in test_docs for s in d['conllu'] for row in s]
gold_spans = list(np.cumsum([len(t) for t in gold_toks]))
# gold_spans = set([(sh-len(gt), sh) for sh, gt in zip(gold_spans, gold_toks)])
gold_spans_nsp = {(sh-len(gt), sh): 1 for sh, gt in zip(gold_spans, gold_toks)}
gold_spans = {(sh-len(gt), sh): 1 for sh, gt in zip(gold_spans, gold_toks)}

confusion = {"TP": 0, "FP": 0, "FN": 0, "P": 0, "R": 0, "F": 0}
confusion_nsp = {"TP": 0, "FP": 0, "FN": 0, "P": 0, "R": 0, "F": 0}
for pred_span, pred_tok in zip(pred_spans, pred_toks):
    if pred_span not in gold_spans:
        confusion['FP'] += 1
        if pred_tok != ' ':
            confusion_nsp['FP'] += 1
    else:
        confusion['TP'] += 1
        if pred_tok != ' ':
            confusion_nsp['TP'] += 1
for gold_span, gold_tok in zip(gold_spans, gold_toks):
    if gold_span not in pred_spans:
        confusion['FN'] += 1
        if gold_tok != ' ':
            confusion_nsp['FN'] += 1
confusion['P'] = confusion['TP']/(confusion['TP'] + confusion['FP']) if (confusion['TP'] + confusion['FP']) else 0
confusion['R'] = confusion['TP']/(confusion['TP'] + confusion['FN']) if (confusion['TP'] + confusion['FN']) else 0
confusion['F'] = 2*confusion['P']*confusion['R']/(confusion['P']+confusion['R']) if (confusion['P']+confusion['R']) else 0
confusion_nsp['P'] = confusion_nsp['TP']/(confusion_nsp['TP'] + confusion_nsp['FP']) if (confusion_nsp['TP'] + confusion_nsp['FP']) else 0
confusion_nsp['R'] = confusion_nsp['TP']/(confusion_nsp['TP'] + confusion_nsp['FN']) if (confusion_nsp['TP'] + confusion_nsp['FN']) else 0
confusion_nsp['F'] = 2*confusion_nsp['P']*confusion_nsp['R']/(confusion_nsp['P']+confusion_nsp['R']) if (confusion_nsp['P']+confusion_nsp['R']) else 0

for d_i, doc in enumerate(model._documents):
    for s_i, s in enumerate(doc._sentences):
        if s._sty is not None:
            result = s._sty == test_docs[d_i]['s_type'][s_i]
            accuracy_sty[test_docs[d_i]['s_type'][s_i]].append(result)
            accuracy_all_sty.append(result)

pred_toks = [t._form for doc in model._documents for s in doc._sentences for t in s._tokens]
pred_arcs = set([(ix, str(t._sup), t._dep, d_i, s_i) for d_i, doc in enumerate(model._documents) 
                 for s_i, s in enumerate(doc._sentences) for ix, t in enumerate(s._tokens)])
pred_spans = list(np.cumsum([len(t) for t in pred_toks]))
pred_stream = [t._pos for doc in model._documents for s in doc._sentences for t in s._tokens]
pred_spans = {(sh-len(gt), sh): (gl, gt)
              for sh, gt, gl in zip(pred_spans, pred_toks, pred_stream)}

gold_toks = [row[1] for d in test_docs for s in d['conllu'] for row in s]
gold_arcs = set([(ix, (str(int(row[6]) - int(row[0])) if int(row[6]) else row[6]), row[7], d_i, s_i) 
                 for d_i, d in enumerate(test_docs) for s_i, s in enumerate(d['conllu']) for ix, row in enumerate(s)])
gold_spans = list(np.cumsum([len(t) for t in gold_toks]))
gold_stream = [row[3] for d in test_docs for s in d['conllu'] for row in s]
gold_spans = {(sh-len(gt), sh): (gl, gt)
              for sh, gt, gl in zip(gold_spans, gold_toks, gold_stream)}

for gold_span in gold_spans:
    if gold_span in pred_spans:
        result = gold_spans[gold_span] == pred_spans[gold_span]
    else:
        result = False
    accuracy[gold_spans[gold_span][0]].append(result)
    accuracy_all.append(result)
    if gold_spans[gold_span][1] != ' ':
        accuracy_nsp[gold_spans[gold_span][0]].append(result)
        accuracy_all_nsp.append(result)
        
for ptok, parc in zip(pred_toks, pred_arcs):
    if parc in gold_arcs:
        sup_accuracy += 1
        if ptok != ' ':
            sup_accuracy_nsp += 1
sup_accuracy /= len(pred_toks)
sup_accuracy_nsp /= len([x for x in pred_toks if x != ' '])

print("Token segmentation performance with/out space", confusion, confusion_nsp)
print("Overall POS accuracy with/out space", sum(accuracy_all)/len(accuracy_all), sum(accuracy_all_nsp)/len(accuracy_all_nsp))
print("Overall SUP:DEP accuracy with/out space", sup_accuracy, sup_accuracy_nsp)
if len(accuracy_all_sty):
    print("Overall s_type accuracy: ", sum(accuracy_all_sty)/len(accuracy_all_sty))
"Tag-wise accuracy", list(Counter({tag: (sum(accuracy_sty[tag])/len(accuracy_sty[tag]), len(accuracy_sty[tag])) 
                                   for tag in accuracy_sty}).most_common()), list(Counter({tag: (sum(accuracy[tag])/len(accuracy[tag]), len(accuracy[tag])) 
                                                                                           for tag in accuracy}).most_common())


0it [00:00, ?it/s]
1it [00:01,  1.12s/it]
2it [00:01,  1.09it/s]
3it [00:02,  1.29it/s]
4it [00:02,  1.48it/s]
5it [00:02,  1.76it/s]
6it [00:03,  1.79it/s]
7it [00:03,  1.84it/s]
8it [00:04,  1.98it/s]
9it [00:04,  2.15it/s]
10it [00:05,  2.00it/s]
11it [00:05,  2.24it/s]
12it [00:05,  2.25it/s]
13it [00:06,  2.03it/s]
14it [00:06,  2.25it/s]
15it [00:07,  2.15it/s]
16it [00:07,  2.31it/s]
17it [00:08,  2.63it/s]
18it [00:08,  2.12it/s]

  0%|          | 0/18 [00:00<?, ?it/s]

Interpreting documents...



100%|██████████| 18/18 [03:28<00:00, 11.61s/it]

Token segmentation performance with/out space {'TP': 11123, 'FP': 6271, 'FN': 5093, 'P': 0.639473381625848, 'R': 0.6859274790330537, 'F': 0.6618863433501935} {'TP': 11123, 'FP': 6271, 'FN': 5093, 'P': 0.639473381625848, 'R': 0.6859274790330537, 'F': 0.6618863433501935}
Overall POS accuracy with/out space 0.39214356191415883 0.39214356191415883
Overall SUP:DEP accuracy with/out space 0.05024721168218926 0.05024721168218926
Overall s_type accuracy:  0.6632302405498282


('Tag-wise accuracy',
 [('decl', (0.9965156794425087, 574)),
  ('intj', (0.19230769230769232, 26)),
  ('frag', (0.02197802197802198, 91)),
  ('imp', (0.0, 49)),
  ('sub', (0.0, 40)),
  ('multiple', (0.0, 31)),
  ('wh', (0.0, 21)),
  ('q', (0.0, 14)),
  ('other', (0.0, 14)),
  ('ger', (0.0, 8)),
  ('inf', (0.0, 5))],
 [('DET', (0.7896995708154506, 1398)),
  ('CCONJ', (0.6377551020408163, 588)),
  ('PUNCT', (0.624757281553398, 2060)),
  ('ADP', (0.6077510275983559, 1703)),
  ('INTJ', (0.38636363636363635, 88)),
  ('NOUN', (0.35846516324469874, 2971)),
  ('PRON', (0.3231197771587744, 1077)),
  ('AUX', (0.29542302357836336, 721)),
  ('ADJ', (0.25316455696202533, 1106)),
  ('SCONJ', (0.2459016393442623, 244)),
  ('PART', (0.21194029850746268, 335)),
  ('VERB', (0.2114114114114114, 1665)),
  ('PROPN', (0.09654062751407884, 1243)),
  ('NUM', (0.01749271137026239, 343)),
  ('ADV', (0.01468189233278956, 613)),
  ('SYM', (0.0, 35)),
  ('X', (0.0, 26))])

In [8]:
from tqdm import tqdm

accuracy = defaultdict(list)
accuracy_nsp = defaultdict(list)
accuracy_all, accuracy_all_nsp, = [], []
sup_accuracy, sup_accuracy_nsp, = 0, 0
accuracy_sty = defaultdict(list)
accuracy_all_sty = []

model.interpret(tdocs, seed = 691, covering = [[[row[1] for row in s] for s in d['conllu']] for d in test_docs])

for d_i, doc in enumerate(model._documents):
    for s_i, s in enumerate(doc._sentences):
        if s._sty is not None:
            result = s._sty == test_docs[d_i]['s_type'][s_i]
            accuracy_sty[test_docs[d_i]['s_type'][s_i]].append(result)
            accuracy_all_sty.append(result)

pred_toks = [t._form for doc in model._documents for s in doc._sentences for t in s._tokens]
pred_arcs = set([(ix, str(t._sup), t._dep, d_i, s_i) for d_i, doc in enumerate(model._documents) 
                 for s_i, s in enumerate(doc._sentences) for ix, t in enumerate(s._tokens)])
pred_spans = list(np.cumsum([len(t) for t in pred_toks]))
pred_stream = [t._pos for doc in model._documents for s in doc._sentences for t in s._tokens]
pred_spans = {(sh-len(gt), sh): (gl, gt)
              for sh, gt, gl in zip(pred_spans, pred_toks, pred_stream)}

gold_toks = [row[1] for d in test_docs for s in d['conllu'] for row in s]
gold_arcs = set([(ix, (str(int(row[6]) - int(row[0])) if int(row[6]) else row[6]), row[7], d_i, s_i) 
                 for d_i, d in enumerate(test_docs) for s_i, s in enumerate(d['conllu']) for ix, row in enumerate(s)])
gold_spans = list(np.cumsum([len(t) for t in gold_toks]))
gold_stream = [row[3] for d in test_docs for s in d['conllu'] for row in s]
gold_spans = {(sh-len(gt), sh): (gl, gt)
              for sh, gt, gl in zip(gold_spans, gold_toks, gold_stream)}

for gold_span in gold_spans:
    if gold_span in pred_spans:
        result = gold_spans[gold_span] == pred_spans[gold_span]
    else:
        result = False
    accuracy[gold_spans[gold_span][0]].append(result)
    accuracy_all.append(result)
    if gold_spans[gold_span][1] != ' ':
        accuracy_nsp[gold_spans[gold_span][0]].append(result)
        accuracy_all_nsp.append(result)
        
for ptok, parc in zip(pred_toks, pred_arcs):
    if parc in gold_arcs:
        sup_accuracy += 1
        if ptok != ' ':
            sup_accuracy_nsp += 1
sup_accuracy /= len(pred_toks)
sup_accuracy_nsp /= len([x for x in pred_toks if x != ' '])

print("Overall POS accuracy with/out space", sum(accuracy_all)/len(accuracy_all), sum(accuracy_all_nsp)/len(accuracy_all_nsp))
print("Overall SUP:DEP accuracy with/out space", sup_accuracy, sup_accuracy_nsp)
if len(accuracy_all_sty):
    print("Overall s_type accuracy: ", sum(accuracy_all_sty)/len(accuracy_all_sty))
"Tag-wise accuracy", list(Counter({tag: (sum(accuracy_sty[tag])/len(accuracy_sty[tag]), len(accuracy_sty[tag])) 
                                   for tag in accuracy_sty}).most_common()), list(Counter({tag: (sum(accuracy[tag])/len(accuracy[tag]), len(accuracy[tag])) 
                                                                                           for tag in accuracy}).most_common())


0it [00:00, ?it/s]
1it [00:01,  1.08s/it]
2it [00:01,  1.12it/s]
3it [00:01,  1.38it/s]
4it [00:02,  1.56it/s]
5it [00:02,  1.81it/s]
6it [00:03,  1.86it/s]
7it [00:03,  1.87it/s]
8it [00:04,  1.98it/s]
9it [00:04,  2.12it/s]
10it [00:05,  1.96it/s]
11it [00:05,  2.20it/s]
12it [00:05,  2.22it/s]
13it [00:06,  2.06it/s]
14it [00:06,  2.24it/s]
15it [00:07,  2.12it/s]
16it [00:07,  2.29it/s]
17it [00:07,  2.60it/s]
18it [00:08,  2.13it/s]

  0%|          | 0/18 [00:00<?, ?it/s]

Interpreting documents...



100%|██████████| 18/18 [03:24<00:00, 11.38s/it]

Overall POS accuracy with/out space 0.5381721756290084 0.5381721756290084
Overall SUP:DEP accuracy with/out space 0.15696812453669384 0.15696812453669384
Overall s_type accuracy:  0.6655480984340044


('Tag-wise accuracy',
 [('decl', (0.9966044142614601, 589)),
  ('intj', (0.19230769230769232, 26)),
  ('frag', (0.021505376344086023, 93)),
  ('imp', (0.02040816326530612, 49)),
  ('sub', (0.0, 41)),
  ('multiple', (0.0, 32)),
  ('wh', (0.0, 21)),
  ('q', (0.0, 16)),
  ('other', (0.0, 14)),
  ('ger', (0.0, 8)),
  ('inf', (0.0, 5))],
 [('DET', (0.8626609442060086, 1398)),
  ('ADP', (0.7480916030534351, 1703)),
  ('CCONJ', (0.6581632653061225, 588)),
  ('PART', (0.6298507462686567, 335)),
  ('PUNCT', (0.616504854368932, 2060)),
  ('PRON', (0.6137418755803157, 1077)),
  ('NOUN', (0.6045102659037361, 2971)),
  ('AUX', (0.5644937586685159, 721)),
  ('INTJ', (0.4431818181818182, 88)),
  ('VERB', (0.439039039039039, 1665)),
  ('ADJ', (0.40596745027124775, 1106)),
  ('SCONJ', (0.3483606557377049, 244)),
  ('PROPN', (0.14400643604183427, 1243)),
  ('ADV', (0.04567699836867863, 613)),
  ('NUM', (0.011661807580174927, 343)),
  ('SYM', (0.0, 35)),
  ('X', (0.0, 26))])